In [1]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
from paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR/'tabular_data.parquet')

df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73135,3.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,3.0,...,29.0,15.0,4.0,12.0,7.0,11.0,4.0,2022-10-27,265,12.0
73136,9.0,4.0,3.0,3.0,2.0,4.0,2.0,6.0,1.0,3.0,...,29.0,17.0,12.0,4.0,5.0,8.0,9.0,2022-10-28,265,3.0
73137,7.0,6.0,3.0,4.0,3.0,5.0,7.0,6.0,5.0,10.0,...,29.0,13.0,9.0,10.0,5.0,10.0,7.0,2022-10-29,265,6.0
73138,6.0,5.0,8.0,6.0,6.0,0.0,1.0,2.0,8.0,6.0,...,8.0,10.0,7.0,3.0,3.0,6.0,2.0,2022-10-30,265,10.0


In [2]:
from datetime import datetime
from data_split import train_test_split

X_train, y_train, X_test, y_test =  train_test_split(
    df, 
    cutoff_date = datetime(2022, 7, 1, 0,0,0),
    target_column_name = 'target_rides_next_hour'
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((40545, 674), (40545,), (32595, 674), (32595,))

In [3]:
#use only past data and remove hour data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]
X_train_only_numeric.shape

(40545, 672)

In [12]:
pip uninstall lightgbm


796.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Found existing installation: lightgbm 3.3.5
Uninstalling lightgbm-3.3.5:
  Would remove:
    /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightgbm-3.3.5.dist-info/*
    /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightgbm/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --no-binary :all: lightgbm


6888.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
  Using cached lightgbm-3.3.5.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: lightgbm is being installed using the legacy 'setup.py install' method, because the '--no-binary' option was enabled for it and this currently disables local wheel building for projects that don't have a 'pyproject.toml' file. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com

In [17]:
import lightgbm as lgb

model = lgb.LGBMRegressor()

model.fit(X_train_only_numeric, y_train)


LGBMRegressor()

In [18]:
from sklearn.metrics import mean_absolute_error

X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
mae_xgb = mean_absolute_error(y_test, predictions)

print(f'{mae_xgb=:.4f}')

mae_xgb=2.4612
